<a href="https://colab.research.google.com/github/gmshroff/metaLearning2022/blob/main/project_data_code/marketEnvForRLprojects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q --upgrade --force-reinstall --no-deps kaggle

In [ ]:
from google.colab import files

In [ ]:
files.upload()

In [ ]:
!mkdir /root/.kaggle

In [ ]:
!mv ./kaggle.json /root/.kaggle/.

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list --user gmshroff

In [ ]:
!kaggle datasets download -d gmshroff/market-data-rl

In [ ]:
!unzip market-data-rl

In [ ]:
!ls

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
class marketEnv():
    def __init__(self,df,symbol,day,target,stoploss,txcost=.001):
        self.df=df.loc[(df['day']==day)&(df['sym']==symbol)]
        self.symbol=symbol
        self.day=day
        self.t=target
        self.l=stoploss
        self.sl=stoploss
        self.time=0
        self.end=self.df.shape[0]
        self.r=self.df['Close'].values
        self.done=False
        self.mv=self.df.iloc[0]['Open']
        self.txcost=txcost
    def thresh(self,x,pos):
        #x=current_price,pos=position_taken(+-)
        if pos>0:
            if x>pos+self.t*pos or x<pos-self.l*pos: return 1
            else: return 0
        elif pos<0:
            if x<abs(pos)-self.t*abs(pos) or x>abs(pos)+self.l*abs(pos): return 1
            else: return 0
    def partial_thresh(self,x,pos):
        #x=current_price,pos=position_taken(+-)
        if pos>0:
            if x>pos+self.t*pos/2: return 1
            else: return 0
        elif pos<0:
            if x<abs(pos)-self.t*abs(pos)/2: return 1
            else: return 0
    def get_state(self):
        return self.df.iloc[0:self.time+1]
    def step(self,action):
        if action==0: 
            ret=0.0
            if self.time+1==self.end: self.done=True
            else: self.time+=1
            return self.df.iloc[0:self.time+1],ret,self.done
        else: 
            r=self.r
            pos=action*r[self.time]
            while True:
                thresh_met=self.thresh(self.r[self.time],pos)
                if thresh_met: break
                if self.time+1==self.end:
                    self.done=True
                    break
                else: self.time+=1
                if self.partial_thresh(self.r[self.time],pos): self.l=-self.t/2
            if pos>0: ret=r[self.time]-pos-r[self.time]*self.txcost
            elif pos<0: ret=abs(pos)-r[self.time]-r[self.time]*self.txcost
            ret=ret*100/self.mv
            self.l=self.sl
            return self.df.iloc[0:self.time+1],ret,self.done

In [ ]:
def run_episode(env,policy):
    env.time=0
    state=env.get_state()
    rewards=[]
    tot=0.0
    done=False
    while done==False:
        action=policy(state)
        state,rew,done=env.step(action)
        tot+=rew
        rewards+=[rew]
        #print(state.shape[0],rew,tot)
    return tot,rewards

In [ ]:
def always_buy(df):
    return 1
def always_sell(df):
    return -1

In [ ]:
dfrl=pd.read_csv('./dfrl.csv')

In [ ]:
# The symbols in each group are those of securities in related industries
groups={0: [189, 167],
 1: [269, 236, 251],
 2: [116, 280, 90],
 3: [137, 31, 231],
 4: [133, 185, 23]}

In [ ]:
symbols=groups[0]

In [ ]:
days=dfrl['day'].unique()

In [ ]:
env=marketEnv(df=dfrl,symbol=symbols[0],day=days[10],target=0.005,stoploss=0.05)

In [ ]:
ret,_=run_episode(env,always_buy)

In [ ]:
ret

In [ ]:
tot_ret=0
print(symbols[0])
for day in days:
    env=marketEnv(df=dfrl,symbol=symbols[0],day=day,target=0.005,stoploss=0.05)
    ret,_=run_episode(env,always_sell)
    tot_ret+=ret
    print(ret,tot_ret)

In [ ]:
env.get_state().shape

In [ ]:
env.step(0)